# Deutsch algorithm（Overview）

## Oracle
Let's start with the oracle used in this algorithm.  In simple terms, it is a function that returns 0 or 1 for a given input.  
As an example, let's consider a situation where you are buying something.   
The output is 1 if the input can be bought at the butcher shop, and 0 if it cannot.

Then,

$$
f(x) = \begin{cases}
   1\ \  (x\ \text{can be bought at the butcher shop})\\
    0\ \  (x\ \text{can NOT be bought at the butcher shop})
  \end{cases}
$$

We can't buy PC at the butcher shop, but we can buy beef, so...

$$
f(\text{PC}) = 0,\ \ \ \ f(\text{beef}) = 1
$$

Such a function that outputs 0 or 1 depending on the input is called an oracle.

In quantum computer algorithms, we use oracle to retrieve only a specific state as an answer given an input state as $x$.  
In other words, it is a gate that extracts only a certain state.

## Deutsch algorithm
First, we explain the algorithm that Deutsch came up with.  
The Deutsch algorithm determine the Oracle of a 1-qubit input that is either $\lvert 0\rangle$ or $\lvert 1\rangle$.

When $x =\lvert 0\rangle$ or $\lvert1\rangle$, there are four possible oracles.

$$
f(x) = \begin{cases}
   0\ \  (x= \lvert 0\rangle)\\
    0\ \  (x= \lvert1\rangle)
  \end{cases}
  \mathrm{or}\ 
  \begin{cases}
   0\ \  (x=\lvert 0\rangle)\\
    1\ \  (x=\lvert 1\rangle)
  \end{cases}
    \mathrm{or}\ 
  \begin{cases}
   1\ \  (x=\lvert 0\rangle)\\
    0\ \  (x=\lvert 1\rangle)
  \end{cases}
    \mathrm{or}\ 
  \begin{cases}
   1\ \  (x=\lvert 0\rangle)\\
    1\ \  (x=\lvert 1\rangle)
  \end{cases}
$$

Deutsch algorithm はこの $f$ が $\lvert 0\rangle$ と $\lvert 1\rangle$ で値が同じものなのか、それとも違うものなのかを判定してくれます。

The Deutsch algorithm will determine if this $f$ outputs the same value for inputs $\lvert 0\rangle$ and $\lvert 1\rangle$ or not.

Consider a concrete circuit.

<img src="../img/016/016_3_new.png" width="40%">

$U_f$ is the Oracle.  
For $q_0$ register, the incoming value $x$ is returned as is, and for $q_1$ register, $y\oplus f(x)$ takes $\rm{mod} 2$ of the sum of the incoming value $y$ and $f(x)$.

Now let's check the status of each.

$$
|\psi_1\rangle = H|0\rangle \otimes H|1\rangle = \frac{1}{\sqrt{2}}(\lvert0\rangle + \lvert1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert0\rangle - \lvert1\rangle)
$$

We obtain

$$
|\psi_1\rangle = \frac{1}{2} \lvert0\rangle \otimes (\lvert0\rangle - \lvert1\rangle) + \frac{1}{2} \lvert1\rangle \otimes (\lvert0\rangle - \lvert1\rangle)\ \ \ \ \cdots (\star)
$$

We can see that the sign changes when the first qubit is flippeed ($\lvert0\rangle \leftrightarrow \lvert1\rangle$).  
In this situation, consider $\lvert\psi_2\rangle$ for the four ways $f(x)$ above.

Because $f(x)$ operates to first qubit,

$$
f(x)=0 \to q_1 \text{ do not changes.}
$$

$$
f(x)=1 \to q_1 \text{ flips.}
$$

Therefore, if we look at the equation $(\star)$

$$
f(0)=f(1) \to \text{the two terms have same sign}
$$

$$
f(0)\neq f(1) \to \text{the two terms have opposite sign}
$$

Then,

$$
\begin {align}
\lvert \psi_2 \rangle &= \begin{cases}
\pm \frac{1}{2} \lvert 0 \rangle \otimes (\lvert 0\rangle - \lvert 1\rangle) \pm \frac{1}{2} \lvert 1 \rangle \otimes (\lvert 0\rangle - \lvert 1\rangle)  \ \  \ \ (f(0)= f(1)) \\
\pm \frac{1}{2} \lvert 0 \rangle \otimes (\lvert 0\rangle - \lvert 1\rangle) \mp \frac{1}{2} \lvert 1 \rangle \otimes (\lvert 0\rangle - \lvert 1\rangle)  \ \  \ \ (f(0)\neq f(1)) \end{cases} \\
&= \begin{cases}
\pm H\lvert 0 \rangle \otimes H \lvert 1 \rangle \ \  \ \ (f(0)= f(1)) \\
 \pm H\lvert 1 \rangle \otimes H \lvert 1 \rangle \ \  \ \ (f(0)\neq f(1))
\end{cases}
\end {align}
$$


Finally, operate $H$ gate to $q_0$.  
Then we obtain

$$
\lvert \psi_3 \rangle = \begin{cases}
\pm \lvert 0\rangle \otimes H\lvert 1 \rangle\ \  \ \ (f(0)= f(1)) \\
\pm \lvert 1\rangle \otimes H\lvert 1 \rangle\ \  \ \ (f(0)\neq f(1))
\end{cases}
$$


The sign $\pm$ is ignored in measurement result, so measurement to 0th qubit can determine the oracle.

This is the  explanation of Deutsch algorithm.

Let's implement it using blueqat.

In [7]:
from blueqat import Circuit
import numpy as np

For the input and output values of $f(x)$, four ways oracles were considered.  
This time, we will implement each of the four ways, and the oracle will be chosen randomly by a random number.   

In [191]:
def oracle_00(c):
    c.i[:]
    
def oracle_01(c):
    c.cx[0, 1]
    
def oracle_10(c):
    c.x[0]
    c.cx[0, 1]
    c.x[0]
    
def oracle_11(c):
    c.x[1]
    
def oracle(c):
    f = np.random.rand()
    if f < 0.25:
        oracle_00(c)
        return "f(0) = 0, f(1) = 0"
    elif f < 0.5:
        oracle_01(c)
        return "f(0) = 0, f(1) = 1"
    elif f < 0.75:
        oracle_10(c)
        return "f(0) = 1, f(1) = 0"
    else:
        oracle_11(c)
        return "f(0) = 1, f(1) = 1"

Here, using Deutsch's algorithm, we can determine the oracle with probability 1 from the measurement results.

In [197]:
c = Circuit(2)
c.x[1].h[:]
oracle_str = oracle(c)
c.h[0].m[0]
res = c.run(shots = 128)
print(res)
print("Selected oracle:", oracle_str)

Counter({'10': 128})
Selected oracle: f(0) = 1, f(1) = 0


If the oracle is $f(0) = f(1)$, measurement results are '00' for all samples.  
If the oracle is $f(0) \neq f(1)$, measurement results are '10' for all samples.